In [18]:
from keras.models import Sequential
import pandas as pd
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D,BatchNormalization,Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from numpy import unique
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
df=pd.read_csv(r'C:\Users\pc\Soil_Ferrtlity.csv', encoding='cp1252')

In [19]:
df

,PH,OC,Mg,N,P,K,Zn,Fe,Cu,Mn,Sand,Silt,Clay,CaCO3,CEC,Fertility_Class
0,5.90,11.50,1.01,380,60.00,279,0.48,6.40,0.64,5.70,65.30,6.80,8.90,3.72,2.81,Very High
1,6.23,6.02,1.43,350,55.70,330,0.27,6.40,0.46,5.60,94.40,9.90,13.70,4.61,10.19,Very High
2,5.56,5.02,1.03,364,55.60,265,0.46,6.20,0.56,6.10,84.50,6.90,12.60,3.53,12.32,Very High
3,6.32,6.04,1.06,312,48.00,233,2.75,5.90,0.82,1.80,70.50,5.70,13.80,5.21,7.02,Very High
4,6.50,5.08,3.13,342,41.00,312,0.34,8.30,0.64,8.60,87.50,6.70,5.80,4.80,10.76,Very High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12504,5.63,2.04,4.04,197,10.72,112,0.88,3.54,0.55,6.52,90.81,10.72,8.67,9.14,9.87,Moderate
12505,7.29,0.56,4.41,213,26.62,259,0.08,4.41,0.63,1.94,86.38,5.39,7.42,2.83,12.26,Moderate
12506,6.34,2.69,1.24,103,27.80,106,1.03,3.68,0.47,2.90,78.85,3.51,8.39,11.43,15.83,Moderate
12507,5.46,1.27,1.81,206,15.86,222,1.53,5.52,0.22,2.34,82.83,5.89,9.22,15.43,14.32,Moderate


In [20]:
le = preprocessing.LabelEncoder()
df['Fertility_Class'] = le.fit_transform(df['Fertility_Class'])
df['Fertility_Class'].unique()

array([3, 1, 4, 0, 2])

In [21]:
X = df.iloc[:, :-1].values
print(X)

[[ 5.9  11.5   1.01 ...  8.9   3.72  2.81]
 [ 6.23  6.02  1.43 ... 13.7   4.61 10.19]
 [ 5.56  5.02  1.03 ... 12.6   3.53 12.32]
 ...
 [ 6.34  2.69  1.24 ...  8.39 11.43 15.83]
 [ 5.46  1.27  1.81 ...  9.22 15.43 14.32]
 [ 5.32  2.61  3.74 ...  5.32  4.21 11.76]]


In [22]:
Y = df.iloc[:, -1].values
Y= le.fit_transform(Y)
print(Y)

[3 3 3 ... 2 2 2]


In [23]:
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(Y)

In [24]:
X = X.reshape(X.shape[0], X.shape[1], 1)
print(X.shape)

(12509, 15, 1)


In [25]:
X = df.iloc[:, :-1].values
scaler = MinMaxScaler(feature_range=(0,1))
# fit and transform in one step
normalized = scaler.fit_transform(X)
# inverse transform''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
inverse = scaler.inverse_transform(normalized)
print(normalized)

[[0.3        1.         0.1039261  ... 0.41789216 0.14066631 0.05994898]
 [0.41       0.51718062 0.15242494 ... 0.7120098  0.18773136 0.37372449]
 [0.18666667 0.42907489 0.10623557 ... 0.64460784 0.13061872 0.46428571]
 ...
 [0.44666667 0.22378855 0.13048499 ... 0.38664216 0.5483871  0.61352041]
 [0.15333333 0.09867841 0.19630485 ... 0.4375     0.75991539 0.54931973]
 [0.10666667 0.21674009 0.41916859 ... 0.19852941 0.16657853 0.44047619]]


In [26]:
xtrain, xtest, ytrain, ytest=train_test_split(X, Y, test_size=0.2, random_state=42, shuffle='True')

In [37]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(15,1)))
model.add(BatchNormalization())
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(5, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_14 (Conv1D)          (None, 13, 64)            256       
                                                                 
 batch_normalization_6 (Batc  (None, 13, 64)           256       
 hNormalization)                                                 
                                                                 
 conv1d_15 (Conv1D)          (None, 11, 64)            12352     
                                                                 
 dropout_7 (Dropout)         (None, 11, 64)            0         
                                                                 
 max_pooling1d_7 (MaxPooling  (None, 5, 64)            0         
 1D)                                                             
                                                                 
 flatten_7 (Flatten)         (None, 320)              

In [38]:
history=model.fit(xtrain, ytrain, validation_data=(xtest, ytest), batch_size=64,epochs=30) 

Epoch 1/30
157/157 [==============================] - 3s 13ms/step - loss: 0.8009 - accuracy: 0.6694 - val_loss: 0.8988 - val_accuracy: 0.5116
Epoch 2/30
157/157 [==============================] - 2s 12ms/step - loss: 0.6222 - accuracy: 0.7481 - val_loss: 0.5808 - val_accuracy: 0.7950
Epoch 3/30
157/157 [==============================] - 2s 12ms/step - loss: 0.5793 - accuracy: 0.7628 - val_loss: 0.5370 - val_accuracy: 0.7918
Epoch 4/30
157/157 [==============================] - 2s 12ms/step - loss: 0.5615 - accuracy: 0.7732 - val_loss: 0.5567 - val_accuracy: 0.8054
Epoch 5/30
157/157 [==============================] - 2s 12ms/step - loss: 0.5536 - accuracy: 0.7770 - val_loss: 0.5120 - val_accuracy: 0.8046
Epoch 6/30
157/157 [==============================] - 2s 12ms/step - loss: 0.5075 - accuracy: 0.7934 - val_loss: 0.4670 - val_accuracy: 0.8437
Epoch 7/30
157/157 [==============================] - 2s 12ms/step - loss: 0.4693 - accuracy: 0.8160 - val_loss: 0.4393 - val_accuracy: 0.8149

In [39]:
acc = model.evaluate(xtrain, ytrain)
print("Loss:", acc[0], " Accuracy:", acc[1])

313/313 [==============================] - 1s 3ms/step - loss: 0.1613 - accuracy: 0.9490
Loss: 0.1612800806760788  Accuracy: 0.9490357041358948


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import numpy

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
cvscores = []
for train, test in kfold.split(X, Y):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(15,1)))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(5, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    history= model.fit(X[train], Y[train], epochs=30, batch_size=64, verbose=0)
    # evaluate the model
    scores = model.evaluate(X[test], Y[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))